# STGCN-PyTorch

In [1]:
import wandb
import os
wandb.login(key="2ba3e7324f00984d9cc7150478390d2c569ba8cf")

wandb.init(project="Methembe-GNN2", name='stgcn-final')

wandb: Currently logged in as: thomastshuma43. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Administrator\.netrc


## Packages

In [1]:
import random
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from load_data import *
from utils import *
from stgcn import *

## Random Seed

In [2]:
torch.manual_seed(2333)
torch.cuda.manual_seed(2333)
np.random.seed(2333)
random.seed(2333)
torch.backends.cudnn.deterministic = True

## Device

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## File Path

In [4]:
matrix_path = "dataset/03062021_M_96.csv"
data_path = "dataset/03062021_T_adj_96.csv"
save_path = "save/model.pt"

## Parameters

In [5]:
day_slot = 288
n_train, n_val, n_test = 34, 15, 15

In [6]:
n_his = 24
n_pred = 12
n_route = 96
Ks, Kt = 3, 3
blocks = [[1, 32, 64], [64, 32, 128]]
drop_prob = 0

In [7]:
batch_size = 50
epochs = 50
lr = 1e-3

## Graph

In [8]:
W = load_matrix(matrix_path)
L = scaled_laplacian(W)
Lk = cheb_poly(L, Ks)
Lk = torch.Tensor(Lk.astype(np.float32)).to(device)

## Standardization

In [9]:
train, val, test = load_data(data_path, n_train * day_slot, n_val * day_slot)
scaler = StandardScaler()
train = scaler.fit_transform(train)
val = scaler.transform(val)
test = scaler.transform(test)

## Transform Data

In [10]:
x_train, y_train = data_transform(train, n_his, n_pred, day_slot, device)
x_val, y_val = data_transform(val, n_his, n_pred, day_slot, device)
x_test, y_test = data_transform(test, n_his, n_pred, day_slot, device)

## DataLoader

In [11]:
train_data = torch.utils.data.TensorDataset(x_train, y_train)
train_iter = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
val_data = torch.utils.data.TensorDataset(x_val, y_val)
val_iter = torch.utils.data.DataLoader(val_data, batch_size)
test_data = torch.utils.data.TensorDataset(x_test, y_test)
test_iter = torch.utils.data.DataLoader(test_data, batch_size)

## Loss & Model & Optimizer

In [12]:
loss = nn.MSELoss()
model = STGCN(Ks, Kt, blocks, n_his, n_route, Lk, drop_prob).to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)

## LR Scheduler

In [13]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.7)

In [22]:
def MAPE(v, v_):
    '''
    Mean absolute percentage error.
    :param v: torch.Tensor or float, ground truth.
    :param v_: torch.Tensor or float, prediction.
    :return: float, MAPE averaged over all elements of input.
    '''
    return torch.mean(torch.abs(v_ - v) / (v + 1e-5))

def MAE(v, v_):
    '''
    Mean absolute error.
    :param v: torch.Tensor or float, ground truth.
    :param v_: torch.Tensor or float, prediction.
    :return: float, MAE averaged over all elements of input.
    '''
    return torch.mean(torch.abs(v_ - v))

def RMSE(v, v_):
    '''
    Root mean squared error.
    :param v: torch.Tensor or float, ground truth.
    :param v_: torch.Tensor or float, prediction.
    :return: float, RMSE averaged over all elements of input.
    '''
    return torch.sqrt(torch.mean((v_ - v) ** 2))

def z_inverse(x, mean, std):
    '''
    The inverse of function z_score().
    :param x: np.ndarray, input to be recovered.
    :param mean: float, the value of mean.
    :param std: float, the value of standard deviation.
    :return: np.ndarray, z-score inverse array.
    '''
    return x * std + mean


## Training & Save Model

In [23]:
# min_val_loss = np.inf
# for epoch in range(1, epochs + 1):
#     l_sum, n = 0.0, 0
#     model.train()
#     for x, y in train_iter:
#         y_pred = model(x).view(len(x), -1)
#         l = loss(y_pred, y)
#         optimizer.zero_grad()
#         l.backward()
#         optimizer.step()
#         l_sum += l.item() * y.shape[0]
#         n += y.shape[0]
#     scheduler.step()
#     val_loss = evaluate_model(model, loss, val_iter)
#     if val_loss < min_val_loss:
#         min_val_loss = val_loss
#         torch.save(model.state_dict(), save_path)
#     print("epoch", epoch, ", train loss:", l_sum / n, ", validation loss:", val_loss)

min_val_loss = np.inf
mape_batch = 0
mae_batch = 0
rmse_batch = 0
for epoch in range(1, epochs + 1):
  l_sum, n = 0.0, 0
  model.train()
  # Track variables for MAPE, MAE, RMSE
  total_abs_error = 0.0
  total_squared_error = 0.0

  for x, y in train_iter:
    y_pred = model(x).view(len(x), -1)
    l = loss(y_pred, y)
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
    l_sum += l.item() * y.shape[0]
    n += y.shape[0]
    
    x_std = torch.std(x)
    x_mean = torch.mean(x)
    
    y = z_inverse(y , x_mean , x_std)
    y_pred = z_inverse(y_pred , x_mean , x_std)
    
    mape_batch += MAPE(y, y_pred)
    mae_batch += MAE(y, y_pred)
    rmse_batch += RMSE(y, y_pred)

    # Calculate errors for MAPE, MAE, RMSE
    abs_error = torch.abs(y_pred - y)  # Absolute error
    squared_error = torch.square(y_pred - y)  # Squared error
    total_abs_error += torch.sum(abs_error).item()
    total_squared_error += torch.sum(squared_error).item()

  scheduler.step()
  val_loss = evaluate_model(model, loss, val_iter)

  # Calculate MAPE, MAE, RMSE after training each epoch
  mape_ = mape_batch/n
  mae_ = mae_batch/n
  mae = total_abs_error / n
  rmse = torch.sqrt(torch.tensor(total_squared_error / n))
  rmse_ = rmse_batch/n
  mape = torch.mean(torch.abs(y_pred - y) / torch.clamp(y, min=1e-8)) * 100  # Avoid division by zero
    
#   wandb.log({"Loss": l.item()  , "RMSE": rmse, "MAE": mae , "MAPE": mape }, step=epoch)

  if val_loss < min_val_loss:
    min_val_loss = val_loss
    torch.save(model.state_dict(), save_path)
  print("epoch", epoch, ", train loss:", l_sum / n, ", validation loss:", val_loss)
  print("Train MAE:", mae, ", Train MAPE:", mape.item(), ", Train RMSE:", rmse.item())
  print(f"New MAPE: {mape_*100}% , New MAE: {mae_} , New RMSE: {rmse_}" )


epoch 1 , train loss: 0.24930231358402968 , validation loss: 1.0262732233003038
Train MAE: 34.085936837017854 , Train MAPE: 1006944704.0 , Train RMSE: 5.188429832458496
New MAPE: -9.983803749084473% , New MAE: 0.00712536321952939 , New RMSE: 0.01049849297851324
epoch 2 , train loss: 0.240571929940898 , validation loss: 0.9960759529041443
Train MAE: 33.65615676785868 , Train MAPE: 2192596480.0 , Train RMSE: 5.098296165466309
New MAPE: -13.125747680664062% , New MAE: 0.014172417111694813 , New RMSE: 0.020844973623752594
epoch 3 , train loss: 0.23416191077570503 , validation loss: 1.1693198049099862
Train MAE: 33.23296453408989 , Train MAPE: 2492928768.0 , Train RMSE: 5.020744800567627
New MAPE: -14.048957824707031% , New MAE: 0.021136747673153877 , New RMSE: 0.03103533759713173
epoch 4 , train loss: 0.22756867468966852 , validation loss: 1.0382367642363226
Train MAE: 32.884696702351825 , Train MAPE: 945797696.0 , Train RMSE: 4.951312065124512
New MAPE: -12.515989303588867% , New MAE: 0.0

KeyboardInterrupt: 

## Load Best Model

In [ ]:
best_model = STGCN(Ks, Kt, blocks, n_his, n_route, Lk, drop_prob).to(device)
best_model.load_state_dict(torch.load(save_path))

## Evaluation

In [ ]:
l = evaluate_model(best_model, loss, test_iter)
MAE, MAPE, RMSE = evaluate_metric(best_model, test_iter, scaler)
print("test loss:", l, "\nMAE:", MAE, ", MAPE:", MAPE, ", RMSE:", RMSE)